Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model works
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a model available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Literary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for literary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelty of British women publishing novels during the early 19th century. 

Can we help close a literary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning algorithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [6]:
import numpy as np
import gensim
import os
import re
import spacy

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:
gensim.__version__

'3.8.1'

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chunks

In [3]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [258]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess


# def tokenize(text):
#     return [token for token in simple_preprocess(text) if token not in STOPWORDS]

import string
nlp = spacy.load('en_core_web_lg')

def tokenize(text):
    
    doc = nlp(text)
    
    STOP_WORDS = nlp.Defaults.stop_words.union(set(['said', 'mr', 'mrs', 'mr.', 'mrs.', 'mister', 'miss']))

    return [token.lemma_.strip().lower().translate(str.maketrans('', '', string.punctuation)) for token in doc
            if (token not in STOP_WORDS)
            and (token not in STOPWORDS)
            and (token.is_punct != True) 
            and (token.like_num != True)
            and (token.is_currency != True)
            and (token.is_digit != True)
            and (token.is_space != True)]

In [259]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(text))      
    return data

In [260]:
tokens = gather_data(path)

In [261]:
tokens[0][0:10]

['pron',
 'find',
 'that',
 'pron',
 'be',
 'only',
 'give',
 'pron',
 'fair',
 'companion']

In [262]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

## Follow Along

In [263]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [264]:
len(titles)

813

In [265]:
len(tokens)

813

### Author DataFrame


In [266]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [267]:
df.head()

,tokens
Austen_Emma0026,"[pron, find, that, pron, be, only, give, pron,..."
Austen_Emma0032,"[feel, so, very, much, like, snow, that, if, p..."
CBronte_Villette0086,"[pierce, the, opaque, blackness, there, stand,..."
CBronte_Jane0099,"[by, a, joke, and, of, pron, queer, look, and,..."
CBronte_Villette0092,"[on, the, second, landing, the, first, floor, ..."


In [268]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [269]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [270]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [271]:
df.head()

,tokens,author,book,section
Austen_Emma0026,"[pron, find, that, pron, be, only, give, pron,...",1,Emma,26
Austen_Emma0032,"[feel, so, very, much, like, snow, that, if, p...",1,Emma,32
CBronte_Villette0086,"[pierce, the, opaque, blackness, there, stand,...",0,Villette,86
CBronte_Jane0099,"[by, a, joke, and, of, pron, queer, look, and,...",0,Jane,99
CBronte_Villette0092,"[on, the, second, landing, the, first, floor, ...",0,Villette,92


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our function. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [272]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(text)
                yield tokens

In [273]:
streaming_data = doc_stream(path)

In [274]:
type(streaming_data)

generator

In [275]:
# gather_data => returns a list
# doc_stream => returns a generator

In [276]:
next(streaming_data) # Returns one document at a time from the generator

['pron',
 'find',
 'that',
 'pron',
 'be',
 'only',
 'give',
 'pron',
 'fair',
 'companion',
 'an',
 'account',
 'of',
 'the',
 'yesterday',
 's',
 'party',
 'at',
 'pron',
 'friend',
 'cole',
 's',
 'and',
 'that',
 'pron',
 'be',
 'come',
 'in',
 'pron',
 'for',
 'the',
 'stilton',
 'cheese',
 'the',
 'north',
 'wiltshire',
 'the',
 'butter',
 'the',
 'cellery',
 'the',
 'beet',
 'root',
 'and',
 'all',
 'the',
 'dessert',
 'this',
 'would',
 'soon',
 'have',
 'lead',
 'to',
 'something',
 'well',
 'of',
 'course',
 'be',
 'pron',
 'consoling',
 'reflection',
 'any',
 'thing',
 'interest',
 'between',
 'those',
 'who',
 'love',
 'and',
 'any',
 'thing',
 'will',
 'serve',
 'as',
 'introduction',
 'to',
 'what',
 'be',
 'near',
 'the',
 'heart',
 'if',
 'pron',
 'could',
 'but',
 'have',
 'keep',
 'long',
 'away',
 'pron',
 'now',
 'walk',
 'on',
 'together',
 'quietly',
 'till',
 'within',
 'view',
 'of',
 'the',
 'vicarage',
 'pale',
 'when',
 'a',
 'sudden',
 'resolution',
 'of',
 

### Gensim LDA Topic Modeling

In [277]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [278]:
id2word.token2id['england']

937

In [279]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(0, 1), (45, 1), (368, 1), (937, 3), (1279, 1), (2321, 1), (14109, 1)]

In [280]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
7056


In [281]:
len(id2word.keys())

18995

In [282]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [283]:
len(id2word.keys())

6587

In [284]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [285]:
corpus[345][:10]

[(2, 3),
 (5, 1),
 (9, 1),
 (20, 1),
 (24, 3),
 (33, 1),
 (35, 1),
 (36, 2),
 (39, 1),
 (41, 1)]

In [286]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=7
                  )

In [287]:
lda.print_topics()

[(0,
  '0.013*"mr" + 0.007*"lydia" + 0.006*"should" + 0.006*"more" + 0.006*"will" + 0.006*"may" + 0.006*"darcy" + 0.006*"go" + 0.006*"wickham" + 0.005*"make"'),
 (1,
  '0.009*"will" + 0.008*"know" + 0.008*"mr" + 0.007*"very" + 0.007*"such" + 0.007*"can" + 0.007*"may" + 0.007*"if" + 0.007*"or" + 0.007*"think"'),
 (2,
  '0.014*"bessie" + 0.008*"miss" + 0.006*"go" + 0.006*"or" + 0.006*"then" + 0.006*"little" + 0.006*"reed" + 0.006*"like" + 0.006*"up" + 0.005*"when"'),
 (3,
  '0.009*"or" + 0.009*"m" + 0.005*"paul" + 0.005*"make" + 0.005*"when" + 0.005*"think" + 0.005*"will" + 0.005*"know" + 0.005*"very" + 0.005*"there"'),
 (4,
  '0.007*"than" + 0.006*"or" + 0.006*"some" + 0.005*"when" + 0.005*"day" + 0.005*"more" + 0.005*"own" + 0.005*"know" + 0.005*"see" + 0.004*"only"'),
 (5,
  '0.007*"or" + 0.006*"like" + 0.006*"there" + 0.006*"seem" + 0.004*"time" + 0.004*"into" + 0.004*"these" + 0.004*"when" + 0.004*"some" + 0.004*"up"'),
 (6,
  '0.009*"little" + 0.008*"or" + 0.008*"bretton" + 0.007*"

In [288]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [289]:
topics = [' '.join(t[0:5]) for t in words]

In [290]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
mr lydia should more will

------ Topic 1 ------
will know mr very such

------ Topic 2 ------
bessie miss go or then

------ Topic 3 ------
or m paul make when

------ Topic 4 ------
than or some when day

------ Topic 5 ------
or like there seem time

------ Topic 6 ------
little or bretton graham think

------ Topic 7 ------
now see or there will

------ Topic 8 ------
elinor marianne mrs very every

------ Topic 9 ------
elizabeth mr darcy bingley very

------ Topic 10 ------
very mr emma mrs think

------ Topic 11 ------
see very will weigh distraction

------ Topic 12 ------
john or dr de now

------ Topic 13 ------
or like look well when

------ Topic 14 ------
will now see go mr



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [291]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [292]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.101524 -0.004219       1        1  17.932386
1     -0.081400  0.025404       2        1  17.070818
14     0.021000 -0.048283       3        1  14.770776
13     0.066839 -0.035206       4        1  13.475638
8     -0.123566 -0.012108       5        1   9.488227
3      0.047744 -0.022981       6        1   7.109561
6      0.031455 -0.058784       7        1   6.291896
9     -0.156566 -0.007347       8        1   5.677210
7      0.048170 -0.027980       9        1   4.338088
2      0.058610 -0.066815      10        1   1.286713
5      0.069578 -0.031905      11        1   1.043859
4      0.013719  0.024979      12        1   0.678174
12     0.048530  0.031873      13        1   0.483067
0     -0.060587  0.093159      14        1   0.341145
11     0.117998  0.140214      15        1   0.012443, topic_info=           Term         Freq        Total Category  logprob  loglift
215          mr  2771.000000  2771.000000  Default  30.0000  30.0000
241          or  2887.000000  2887.000000  Default  29.0000  29.0000
1590  elizabeth   674.000000   674.000000  Default  28.0000  28.0000
216         mrs  1975.000000  1975.000000  Default  27.0000  27.0000
212        more  2074.000000  2074.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
218        must     0.045911  1816.280396  Topic15  -7.0725  -1.5938
139          go     0.046861  2036.798462  Topic15  -7.0521  -1.6879
400         can     0.043551  1776.576782  Topic15  -7.1253  -1.6245
479        like     0.042348  1571.786133  Topic15  -7.1533  -1.5300
318        take     0.042113  1553.862427  Topic15  -7.1589  -1.5241

[1174 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.966640       abbey
5712      1  0.038811       abbot
5712      3  0.426921       abbot
5712     10  0.465732       abbot
6042      2  0.886366  abominably
...     ...       ...         ...
574      14  0.002677       young
4540      4  0.737493     zoraide
4540      6  0.070237     zoraide
4540      7  0.035119     zoraide
4540      9  0.140475     zoraide

[3881 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 2, 15, 14, 9, 4, 7, 10, 8, 3, 6, 5, 13, 1, 12])

### Overall Model / Documents

In [294]:
lda[corpus[0]]

[(1, 0.09602889), (10, 0.89574224)]

In [295]:
distro = [lda[d] for d in corpus]

In [296]:
distro[0]

[(1, 0.09694017), (10, 0.89503443)]

In [300]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [301]:
len(new_distro)

813

In [302]:
df.head()

,tokens,author,book,section
Austen_Emma0026,"[pron, find, that, pron, be, only, give, pron,...",1,Emma,26
Austen_Emma0032,"[feel, so, very, much, like, snow, that, if, p...",1,Emma,32
CBronte_Villette0086,"[pierce, the, opaque, blackness, there, stand,...",0,Villette,86
CBronte_Jane0099,"[by, a, joke, and, of, pron, queer, look, and,...",0,Jane,99
CBronte_Villette0092,"[on, the, second, landing, the, first, floor, ...",0,Villette,92


In [303]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [304]:
df.head()

,mr lydia should more will,will know mr very such,bessie miss go or then,or m paul make when,than or some when day,or like there seem time,little or bretton graham think,now see or there will,elinor marianne mrs very every,elizabeth mr darcy bingley very,very mr emma mrs think,see very will weigh distraction,john or dr de now,or like look well when,will now see go mr,author
Austen_Emma0026,0.0,0.098169,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.900260,0,0.0,0.000000,0.000000,Austen
Austen_Emma0032,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.998367,0,0.0,0.000000,0.000000,Austen
CBronte_Villette0086,0.0,0.012618,0.0,0.000000,0.0,0.0,0.647484,0.281435,0.0,0.000000,0.000000,0,0.0,0.000000,0.057122,CBronte
CBronte_Jane0099,0.0,0.000000,0.0,0.049929,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0,0.0,0.174336,0.774162,CBronte
CBronte_Villette0092,0.0,0.073654,0.0,0.000000,0.0,0.0,0.285862,0.271943,0.0,0.019797,0.042856,0,0.0,0.304853,0.000000,CBronte


In [305]:
df.groupby('author').mean()

,mr lydia should more will,will know mr very such,bessie miss go or then,or m paul make when,than or some when day,or like there seem time,little or bretton graham think,now see or there will,elinor marianne mrs very every,elizabeth mr darcy bingley very,very mr emma mrs think,see very will weigh distraction,john or dr de now,or like look well when,will now see go mr
author,,,,,,,,,,,,,,,
Austen,0.007444,0.262751,0.000071,0.000373,0.005930,0.000000,0.000509,0.000428,0.201732,0.129971,0.374619,0,0.000000,0.004005,0.009763
CBronte,0.000198,0.105748,0.023774,0.125005,0.006647,0.018634,0.109334,0.077371,0.001177,0.000510,0.005531,0,0.008413,0.239179,0.275939


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text preprocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=7)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=10)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)